In [ ]:
import os

In [1]:
directory = "results/classifications/20220217/"
suffix= '20220217'

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=30 "\
       "--config GDAL_CACHEMAX 512 "
       + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

## Restart the kernel here to reset the directory

## Rerun suffix cell

## Clip mosaic to AEZ

In [2]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:43223 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


In [4]:
aez_region = 'data/Southeast.shp'
# aez_region = 'data/zambia.shp'
pred_tif = "results/classifications/"+suffix+"/Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':5000, 'y':5000}

In [5]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [6]:
aez = xr_rasterize(aez, da)

/env/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [8]:
%%time
da = da.compute()

CPU times: user 12.6 s, sys: 30.4 s, total: 42.9 s
Wall time: 1min 18s


In [9]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True)

PosixPath('results/classifications/20220217/Southeast_gm_mads_two_seasons_20220217_mosaic_clipped.tif')

In [ ]:
# write_cog(aez.squeeze(),
#           'WDPA_southern.tif',
#           overwrite=True)

In [ ]:
# write_cog(da.squeeze(),
#           "Zambia_cropmask_"+suffix+".tif",
#           overwrite=True)